## 3.6 Homework

The goal of this homework is to familiarize users with workflow orchestration. We start from the solution of homework 1. The notebook can be found below:

https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/01-intro/homework.ipynb

This has already been converted to a script called `homework.py` in the `03-orchestration` folder of this repo. 

You will use the FHV dataset like in homework 1.


## Motivation

We already have a model training script. Maybe a data scientist in your team handed it to you and your job is schedule the running of training script using a workflow orchestration - Prefect in this case. Below are the requirements. Do not implement them yet, we will do so in this exercise. Just understand the goal.

1. The training flow will be run every month.
2. The flow will take in a parameter called `date` which will be a datetime.
    a. `date` should default to None
    b. If `date` is None, set `date` as the current day. Use the data from 2 months back as the training data and the data from the previous month as validation data.
    c. If `date` is passed, get 2 months before the `date` as the training data, and the previous month as validation data.
    d. As a concrete example, if the date passed is "2021-03-15", the training data should be "fhv_tripdata_2021-01.parquet" and the validation file will be "fhv_trip_data_2021-02.parquet"
3. Save the model as "model-{date}.pkl" where date is in `YYYY-MM-DD`. Note that `date` here is the value of the flow `parameter`. In practice, this setup makes it very easy to get the latest model to run predictions because you just need to get the most recent one.
4. In this example we use a DictVectorizer. That is needed to run future data through our model. Save that as "dv-{date}.pkl". Similar to above, if the date is `2021-03-15`, the files output should be `model-2021-03-15.bin` and `dv-2021-03-15.b`.

This convention is not strict in industry, and in practice, you will come up with your own system to manage these training pipeline runs. For example, if we wanted to train on the whole history instead of just one month, we'd need to allow for added parameterization and logic in our flow. If the data came in weekly instead of monthly, we might need a different naming convention. But these requirements are already a simple approximation of something you could use in production.

On the deployment side, it's very easy to just pull in the latest data and predict it using the latest model and vectorizer files. Tools the MLFlow in the last chapter can simplify that process as well. This homework will focus more on the batch training.

In order, this homework assignment will be about:

1. Converting the script to a Flow
2. Changing the parameters to take in a `date`. Making this parameter dynamic.
3. Scheduling a batch training job that outputs the latest model somewhere

## Setup

You can use either local Prefect Orion or a VM hosted Prefect Orion instance for this. It shouldn't matter. Just note that if you use a VM hosted one, you will need to configure your local API to hit the VM.

Video 3.4 of the course will give more detailed instructions if you been run it on a VM.

## Q1. Converting the script to a Prefect flow

If you want to follow the videos exactly, do:

```bash
pip install prefect==2.0b5
```

If you need Windows support, check [`windows.md`](windows.md) for installation instructions.

The current script `homework.py` is a fully functional script as long as you already have `fhv_trip_data_2021-01.parquet` and `fhv_trip_data_2021-02.parquet` inside a `data` folder. You should be able to already run it using:

```bash
python homework.py
```

We want to bring this to workflow orchestration to add observability around it. The `main` function will be converted to a `flow` and the other functions will be `tasks`. After adding all of the decorators, there is actually one task that you will need to call `.result()` for inside the `flow` to get it to work. Which task is this?

* `read_data`
* `prepare_features`
* **`train_model`**
* `run_model`

Important: change all `print` statements to use the Prefect logger. Using the `print` statement will not appear in the Prefect UI. You have to call `get_run_logger` at the start of the task to use it.

**ANSWER:** `train_model`

**Note**: In this case we need use ``.result()`` in the ``train_model`` because inside ``train_model`` we are mixing normal functions(like **mean_squared_error** and **DictVectorizer()** class) with the output of the function **prepare_features** function wrapped around a **@task**

In [1]:
%%writefile homework_modified.py
import pandas as pd

from prefect import flow, task, get_run_logger
from prefect.task_runners import SequentialTaskRunner

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

@task
def read_data(path):
    df = pd.read_parquet(path)
    return df

@task
def prepare_features(df, categorical, train=True):
    logger = get_run_logger()
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    mean_duration = df.duration.mean()
    if train:
        logger.info(f"The mean duration of training is {mean_duration}")
    else:
        logger.info(f"The mean duration of validation is {mean_duration}")
    
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    return df

@task
def train_model(df, categorical):
    logger = get_run_logger()
    
    train_dicts = df[categorical].to_dict(orient='records')
    dv = DictVectorizer()
    X_train = dv.fit_transform(train_dicts) 
    y_train = df.duration.values
    
    logger.info(f"The shape of X_train is {X_train.shape}")
    logger.info(f"The DictVectorizer has {len(dv.feature_names_)} features")

    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_train)
    mse = mean_squared_error(y_train, y_pred, squared=False)
    logger.info(f"The MSE of training is: {mse}")
    return lr, dv

@task
def run_model(df, categorical, dv, lr):
    logger = get_run_logger()
    val_dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(val_dicts) 
    y_pred = lr.predict(X_val)
    y_val = df.duration.values

    mse = mean_squared_error(y_val, y_pred, squared=False)
    logger.info(f"The MSE of validation is: {mse}")
    return

@flow(task_runner=SequentialTaskRunner())
def main(train_path: str = './data/fhv_tripdata_2021-01.parquet', 
           val_path: str = './data/fhv_tripdata_2021-02.parquet'):

    categorical = ['PUlocationID', 'DOlocationID']

    df_train = read_data(train_path)
    df_train_processed = prepare_features(df_train, categorical)

    df_val = read_data(val_path)
    df_val_processed = prepare_features(df_val, categorical, False)

    # train the model
    lr, dv = train_model(df_train_processed, categorical).result()
    run_model(df_val_processed, categorical, dv, lr)

main()

Overwriting homework_modified.py


In [2]:
!python homework_modified.py

18:27:27.372 | INFO    | prefect.engine - Created flow run 'aquamarine-carp' for flow 'main'
18:27:27.372 | INFO    | Flow run 'aquamarine-carp' - Using task runner 'SequentialTaskRunner'
18:27:27.385 | WARNING | Flow run 'aquamarine-carp' - No default storage is configured on the server. Results from this flow run will be stored in a temporary directory in its runtime environment.
18:27:27.449 | INFO    | Flow run 'aquamarine-carp' - Created task run 'read_data-4c7f9de4-0' for task 'read_data'
18:27:35.121 | INFO    | Task run 'read_data-4c7f9de4-0' - Finished in state Completed()
18:27:35.161 | INFO    | Flow run 'aquamarine-carp' - Created task run 'prepare_features-4ee39d9f-0' for task 'prepare_features'
18:27:35.376 | INFO    | Task run 'prepare_features-4ee39d9f-0' - The mean duration of training is 16.2472533682457
18:27:46.858 | INFO    | Task run 'prepare_features-4ee39d9f-0' - Finished in state Completed()
18:27:46.893 | INFO    | Flow run 'aquamarine-carp' - Created task run


## Q2. Parameterizing the flow

Right now there are two parameters for `main()` called `train_path` and `val_path`. We want to change the flow function to accept `date` instead. `date` should then be passed to a task that gives both the `train_path` and `val_path` to use.

It should look like this:

```python
@flow
def main(date=None):
    train_path, val_path = get_paths(date).result()
    # rest of flow below
```

Where `get_paths` is a task that you have to implement. The specs for this are outlined in the motivation section. Listing them out again here:

The flow will take in a parameter called `date` which will be a datetime.
    a. `date` should default to None
    b. If `date` is None, use the current day. Use the data from 2 months back as the training data and the data from the previous month as validation data.
    c. If a `date` value is supplied, get 2 months before the `date` as the training data, and the previous month as validation data.
    d. As a concrete example, if the date passed is "2021-03-15", the training data should be "fhv_tripdata_2021-01.parquet" and the validation file will be "fhv_trip_data_2021-02.parquet"

Because we have two files:

* fhv_tripdata_2021-01.parquet
* fhv_tripdata_2021-02.parquet

Change the `main()` flow call to the following:

```python
main(date="2021-03-15")
```

and it should use those files. This is a simplification for testing our homework.

Recall the page from where we downloaded the For-Hire trip data.

https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

Download the relevant files needed to run the `main` flow if `date` is 2021-08-15.

For example:

```python
main(date="2021-08-15")
```

By setting up the logger from the previous step, we should see some logs about our training job. What is the validation MSE when running the flow with this date?

Note you need to download the relevant files to run. Part of this question is understanding which files the flow should be looking for.

The validation MSE is:

* **11.637**
* 11.837
* 12.037
* 12.237

**ANSWER**: 11.637028964002573

In [18]:
%%writefile homework_modified.py
import pandas as pd

from datetime import datetime

from prefect import flow, task, get_run_logger
from prefect.task_runners import SequentialTaskRunner

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

@task
def get_paths(date: datetime = None) -> list[str, str]:
    if date == None:
        date_pivot = datetime.now()
    else:
        date = datetime.strptime(date, '%Y-%m-%d')
        date_pivot = date
    
    date_train = date_pivot - pd.DateOffset(months = 2)
    date_val = date_pivot - pd.DateOffset(months = 1)    
    train_path = "./data/fhv_tripdata_{:04d}-{:02d}.parquet".format(date_train.year, date_train.month)
    val_path = "./data/fhv_tripdata_{:04d}-{:02d}.parquet".format(date_val.year, date_val.month)
    
    return train_path, val_path

@task
def read_data(path):
    df = pd.read_parquet(path)
    return df

@task
def prepare_features(df, categorical, train=True):
    logger = get_run_logger()
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    mean_duration = df.duration.mean()
    if train:
        logger.info(f"The mean duration of training is {mean_duration}")
    else:
        logger.info(f"The mean duration of validation is {mean_duration}")
    
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    return df

@task
def train_model(df, categorical):
    logger = get_run_logger()
    
    train_dicts = df[categorical].to_dict(orient='records')
    dv = DictVectorizer()
    X_train = dv.fit_transform(train_dicts) 
    y_train = df.duration.values
    
    logger.info(f"The shape of X_train is {X_train.shape}")
    logger.info(f"The DictVectorizer has {len(dv.feature_names_)} features")

    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_train)
    mse = mean_squared_error(y_train, y_pred, squared=False)
    logger.info(f"The MSE of training is: {mse}")
    return lr, dv

@task
def run_model(df, categorical, dv, lr):
    logger = get_run_logger()
    val_dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(val_dicts) 
    y_pred = lr.predict(X_val)
    y_val = df.duration.values

    mse = mean_squared_error(y_val, y_pred, squared=False)
    logger.info(f"The MSE of validation is: {mse}")
    return

@flow(task_runner=SequentialTaskRunner())
def main(date):
    train_path, val_path = get_paths(date).result()
    
    categorical = ['PUlocationID', 'DOlocationID']

    df_train = read_data(train_path)
    df_train_processed = prepare_features(df_train, categorical)

    df_val = read_data(val_path)
    df_val_processed = prepare_features(df_val, categorical, False)

    # train the model
    lr, dv = train_model(df_train_processed, categorical).result()
    run_model(df_val_processed, categorical, dv, lr)


main(date = "2021-08-15")

Overwriting homework_modified.py


In [19]:
!python homework_modified.py

19:20:39.997 | INFO    | prefect.engine - Created flow run 'radiant-spaniel' for flow 'main'
19:20:39.997 | INFO    | Flow run 'radiant-spaniel' - Using task runner 'SequentialTaskRunner'
19:20:40.007 | WARNING | Flow run 'radiant-spaniel' - No default storage is configured on the server. Results from this flow run will be stored in a temporary directory in its runtime environment.
19:20:40.064 | INFO    | Flow run 'radiant-spaniel' - Created task run 'get_paths-6e696e34-0' for task 'get_paths'
19:20:40.128 | INFO    | Task run 'get_paths-6e696e34-0' - Finished in state Completed()
19:20:40.158 | INFO    | Flow run 'radiant-spaniel' - Created task run 'read_data-4c7f9de4-0' for task 'read_data'
19:20:48.905 | INFO    | Task run 'read_data-4c7f9de4-0' - Finished in state Completed()
19:20:48.940 | INFO    | Flow run 'radiant-spaniel' - Created task run 'prepare_features-4ee39d9f-0' for task 'prepare_features'
19:20:49.169 | INFO    | Task run 'prepare_features-4ee39d9f-0' - The mean dur

## Q3. Saving the model and artifacts

At the moment, we are not saving the model and vectorizer for future use. You don't need a new task for this, you can just add it inside the `flow`. The requirements for filenames to save it as were mentioned in the Motivation section. They are pasted again here:

* Save the model as "model-{date}.pkl" where date is in `YYYY-MM-DD`. Note that `date` here is the value of the flow `parameter`. In practice, this setup makes it very easy to get the latest model to run predictions because you just need to get the most recent one.
* In this example we use a DictVectorizer. That is needed to run future data through our model. Save that as "dv-{date}.pkl". Similar to above, if the date is `2021-03-15`, the files output should be `model-2021-03-15.bin` and `dv-2021-03-15.b`.

By using this file name, during inference, we can just pull the latest model from our model directory and apply it. Assuming we already had a list of filenames:

```python
['model-2021-03-15.bin', 'model-2021-04-15.bin', 'model-2021-05-15.bin']
```

We could do something like `sorted(model_list, reverse=False)[0]` to get the filename of the latest file. This is the simplest way to consistently use the latest trained model for inference. Tools like MLFlow give us more control logic to use flows.

What is the file size of the `DictVectorizer` that we trained when the `date` is 2021-08-15?

* **13,000 bytes** 
* 23,000 bytes 
* 33,000 bytes 
* 43,000 bytes 

You can bring up the Orion UI and see the work you've been doing. If you are using local Orion, you can start the server with:

```bash
prefect orion start
```

You should be able to see previous Flow runs and the most recent successful runs. Navigate to some of them. Take time to explore the UI. The radar plot would be a good thing to share on social media if you participate in those posts.

**ANSWER**: 13.2kB

In [34]:
%%writefile homework_modified.py
import pickle
import pandas as pd

from datetime import datetime

from prefect import flow, task, get_run_logger
from prefect.task_runners import SequentialTaskRunner

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

@task
def get_paths(date: datetime = None) -> list[str, str]:
    if date == None:
        date_pivot = datetime.now()
    else:
        date = datetime.strptime(date, '%Y-%m-%d')
        date_pivot = date
    
    date_train = date_pivot - pd.DateOffset(months = 2)
    date_val = date_pivot - pd.DateOffset(months = 1)    
    train_path = "./data/fhv_tripdata_{:04d}-{:02d}.parquet".format(date_train.year, date_train.month)
    val_path = "./data/fhv_tripdata_{:04d}-{:02d}.parquet".format(date_val.year, date_val.month)
    
    return train_path, val_path

@task
def read_data(path):
    df = pd.read_parquet(path)
    return df

@task
def prepare_features(df, categorical, train=True):
    logger = get_run_logger()
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    mean_duration = df.duration.mean()
    if train:
        logger.info(f"The mean duration of training is {mean_duration}")
    else:
        logger.info(f"The mean duration of validation is {mean_duration}")
    
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    return df

@task
def train_model(df, categorical):
    logger = get_run_logger()
    
    train_dicts = df[categorical].to_dict(orient='records')
    dv = DictVectorizer()
    X_train = dv.fit_transform(train_dicts) 
    y_train = df.duration.values
    
    logger.info(f"The shape of X_train is {X_train.shape}")
    logger.info(f"The DictVectorizer has {len(dv.feature_names_)} features")

    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_train)
    mse = mean_squared_error(y_train, y_pred, squared=False)
    logger.info(f"The MSE of training is: {mse}")
    return lr, dv

@task
def run_model(df, categorical, dv, lr):
    logger = get_run_logger()
    val_dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(val_dicts) 
    y_pred = lr.predict(X_val)
    y_val = df.duration.values

    mse = mean_squared_error(y_val, y_pred, squared=False)
    logger.info(f"The MSE of validation is: {mse}")
    return

@flow(task_runner=SequentialTaskRunner())
def main(date):
    train_path, val_path = get_paths(date).result()
    
    categorical = ['PUlocationID', 'DOlocationID']

    df_train = read_data(train_path)
    df_train_processed = prepare_features(df_train, categorical)

    df_val = read_data(val_path)
    df_val_processed = prepare_features(df_val, categorical, False)

    # train the model
    lr, dv = train_model(df_train_processed, categorical).result()
    run_model(df_val_processed, categorical, dv, lr)
    
    # Save Model and Artifacts
    with open(f'artifacts/model-{date}.bin', 'wb') as f_out:
        pickle.dump(lr, f_out)
    with open(f'artifacts/dv-{date}.bin', 'wb') as f_out:
        pickle.dump(dv, f_out)

main(date = "2021-08-15")

Overwriting homework_modified.py


In [35]:
!python homework_modified.py

19:56:12.832 | INFO    | prefect.engine - Created flow run 'sassy-donkey' for flow 'main'
19:56:12.832 | INFO    | Flow run 'sassy-donkey' - Using task runner 'SequentialTaskRunner'
19:56:12.843 | WARNING | Flow run 'sassy-donkey' - No default storage is configured on the server. Results from this flow run will be stored in a temporary directory in its runtime environment.
19:56:12.897 | INFO    | Flow run 'sassy-donkey' - Created task run 'get_paths-6e696e34-0' for task 'get_paths'
19:56:12.966 | INFO    | Task run 'get_paths-6e696e34-0' - Finished in state Completed()
19:56:12.998 | INFO    | Flow run 'sassy-donkey' - Created task run 'read_data-4c7f9de4-0' for task 'read_data'
19:56:21.942 | INFO    | Task run 'read_data-4c7f9de4-0' - Finished in state Completed()
19:56:21.983 | INFO    | Flow run 'sassy-donkey' - Created task run 'prepare_features-4ee39d9f-0' for task 'prepare_features'
19:56:22.317 | INFO    | Task run 'prepare_features-4ee39d9f-0' - The mean duration of training 

## Q4. Creating a deployment with a CronSchedule

We previously showed the `IntervalSchedule` in the video tutorials. In some cases, the interval is too rigid. For example, what if we wanted to run this `flow` on the 15th of every month? An interval of 30 days would not be in sync. In cases like these, the `CronSchedule` is more appropriate. The documentation for that is [here](https://orion-docs.prefect.io/concepts/schedules/#cronschedule)

Cron is an important part of workflow orchestration. It is used to schedule tasks, and was a predecessor for more mature orchestration frameworks. A lot of teams still use Cron in production. Even if you don't use Cron, the Cron expression is very common as a way to write a schedule, and the basics are worth learning for orchestration, even outside Prefect.

For this exercise, use a `CronSchedule` when creating a Prefect deployment.

What is the Cron expression to run a flow at 9 AM every 15th of the month?

* `* * 15 9 0`
* `9 15 * * *`
* **`0 9 15 * *`**
* `0 15 9 1 *`

Hint: there are many Cron to English tools. Try looking for one to help you.

Create a deployment with `prefect deployment create` after you write your `DeploymentSpec`

**ANSWER:** ```0 9 15 * *```

In [1]:
%%writefile prefect_deploy_modified.py
import pickle
import pandas as pd

from datetime import datetime

from prefect import flow, task, get_run_logger
from prefect.task_runners import SequentialTaskRunner
from prefect.deployments import DeploymentSpec
from prefect.orion.schemas.schedules import CronSchedule
from prefect.flow_runners import SubprocessFlowRunner

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

@task
def get_paths(date: datetime = None) -> list[str, str]:
    if date == None:
        date_pivot = datetime.now()
    else:
        date = datetime.strptime(date, '%Y-%m-%d')
        date_pivot = date
    
    date_train = date_pivot - pd.DateOffset(months = 2)
    date_val = date_pivot - pd.DateOffset(months = 1)    
    train_path = "./data/fhv_tripdata_{:04d}-{:02d}.parquet".format(date_train.year, date_train.month)
    val_path = "./data/fhv_tripdata_{:04d}-{:02d}.parquet".format(date_val.year, date_val.month)
    
    return train_path, val_path

@task
def read_data(path):
    df = pd.read_parquet(path)
    return df

@task
def prepare_features(df, categorical, train=True):
    logger = get_run_logger()
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    mean_duration = df.duration.mean()
    if train:
        logger.info(f"The mean duration of training is {mean_duration}")
    else:
        logger.info(f"The mean duration of validation is {mean_duration}")
    
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    return df

@task
def train_model(df, categorical):
    logger = get_run_logger()
    
    train_dicts = df[categorical].to_dict(orient='records')
    dv = DictVectorizer()
    X_train = dv.fit_transform(train_dicts) 
    y_train = df.duration.values
    
    logger.info(f"The shape of X_train is {X_train.shape}")
    logger.info(f"The DictVectorizer has {len(dv.feature_names_)} features")

    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_train)
    mse = mean_squared_error(y_train, y_pred, squared=False)
    logger.info(f"The MSE of training is: {mse}")
    return lr, dv

@task
def run_model(df, categorical, dv, lr):
    logger = get_run_logger()
    val_dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(val_dicts) 
    y_pred = lr.predict(X_val)
    y_val = df.duration.values

    mse = mean_squared_error(y_val, y_pred, squared=False)
    logger.info(f"The MSE of validation is: {mse}")
    return

@flow(task_runner=SequentialTaskRunner())
def main(date = "2021-08-15"):
    train_path, val_path = get_paths(date).result()
    
    categorical = ['PUlocationID', 'DOlocationID']

    df_train = read_data(train_path)
    df_train_processed = prepare_features(df_train, categorical)

    df_val = read_data(val_path)
    df_val_processed = prepare_features(df_val, categorical, False)

    # train the model
    lr, dv = train_model(df_train_processed, categorical).result()
    run_model(df_val_processed, categorical, dv, lr)
    
    # Save Model and Artifacts
    with open(f'artifacts/model-{date}.bin', 'wb') as f_out:
        pickle.dump(lr, f_out)
    with open(f'artifacts/dv-{date}.bin', 'wb') as f_out:
        pickle.dump(dv, f_out)

DeploymentSpec(
  flow = main,
  name = "momdel_training_from_h3",
  schedule = CronSchedule(cron = "0 9 15 * *",
                          timezone = "America/New_York"),
  flow_runner = SubprocessFlowRunner(),
  tags = ["ml_from_h3"]
)

Overwriting prefect_deploy_modified.py


In [4]:
!prefect deployment create prefect_deploy_modified.py

Loading deployment specifications from python script at 
'prefect_deploy_modified.py'...
/home/w10/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/prefect/deployments.py:247: UserWarning: You have configured local storage, this deployment will only be usable from the current machine..
  warnings.warn(
Creating deployment 'momdel_training_from_h3' for flow 'main'...
Deploying flow script from 
'/home/w10/homeworks/homework_3/prefect_deploy_modified.py' using Local 
Storage...
Created deployment 'main/momdel_training_from_h3'.
View your new deployment with: 

    prefect deployment inspect 'main/momdel_training_from_h3'
Created 1 deployments!


## Q5. Viewing the Deployment 

View the deployment in the UI. When first loading, we may not see that many flows because the default filter is 1 day back and 1 day forward. Remove the filter for 1 day forward to see the scheduled runs. 

How many flow runs are scheduled by Prefect in advance? You should not be counting manually. There is a number of upcoming runs on the top right of the dashboard.

* 0
* **3**
* 10
* 25

**ANSWER:** 3

## Q6. Creating a work-queue

In order to run this flow, you will need an agent and a work queue. Because we scheduled our flow on every month, it won't really get picked up by an agent. For this exercise, create a work-queue from the UI and view it using the CLI. 

For all CLI commands with Prefect, you can use `--help` to get more information. 

For example,

* `prefect --help`
* `prefect work-queue --help`

What is the command to view the available work-queues?

* `prefect work-queue inspect`
* **`prefect work-queue ls`**
* `prefect work-queue preview`
* `prefect work-queue list`

**ANSWER**: `prefect work-queue ls`

In [10]:
!prefect work-queue --help

Usage: prefect work-queue [OPTIONS] COMMAND [ARGS]...

  Commands for work queue CRUD.

Options:
  --help  Show this message and exit.

Commands:
  clear-concurrency-limit  Clear any concurrency limits from a work queue.
  create                   Create a work queue.
  delete                   Delete a work queue by ID.
  inspect                  Inspect a work queue by ID.
  ls                       View all work queues.
  pause                    Pause a work queue.
  preview                  Preview a work queue.
  resume                   Resume a paused work queue.
  set-concurrency-limit    Set a concurrency limit on a work queue.


In [9]:
!prefect work-queue ls

                             Work Queues                             
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃                                   ID ┃ Name   ┃ Concurrency Limit ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ f57d880c-dc52-4a04-afc4-cd6ac2905744 │ global │ None              │
└──────────────────────────────────────┴────────┴───────────────────┘
                     (**) denotes a paused queue                     
